# Loading Geospatial Data with GeoPandas: Supported Data Types

In [1]:
# Import necessary libraries
import geopandas as gpd
import pandas as pd

In [ ]:

shapefile = "data/Pedestrian Zone Shapefile (Tabular)_20241220/geo_export_a7bb075a-41dc-445f-8244-8430e90a8dde.shp"
shapefile_gdf = gpd.read_file(shapefile)
shapefile_gdf.head()

In [ ]:
shapefile_gdf.explore()

In [4]:
geojson_fp = "https://raw.githubusercontent.com/HodgesWardElliott/custom-nyc-neighborhoods/refs/heads/master/custom-pedia-cities-nyc-Mar2018.geojson"
geojson_gdf = gpd.read_file(geojson_fp)

In [ ]:
geojson_gdf.explore()

In [ ]:
gpkg_fp = "https://ngageoint.github.io/GeoPackage/examples/rivers.gpkg"
gpkg_gdf = gpd.read_file(gpkg_fp)
gpkg_gdf.explore()

In [ ]:
fgb_fp = "https://github.com/flatgeobuf/flatgeobuf/raw/refs/heads/master/test/data/UScounties.fgb"
fgb_gdf = gpd.read_file(fgb_fp)
fgb_gdf.explore()

In [ ]:
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
import duckdb


# Define PostGIS connection parameters
host = "your_host"           # e.g., "localhost" or your database IP
database = "your_database"   # Database name
user = "your_user"           # Username
password = "your_password"   # Password
port = "5432"                # Default PostgreSQL port

# SQLAlchemy connection string for PostGIS
postgis_connection = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create SQLAlchemy engine
engine = create_engine(postgis_connection)

# Define SQL query to read spatial data
# Replace 'your_table_name' with the actual table containing spatial data
query = "SELECT * FROM your_table_name"

# Load the PostGIS table into a GeoDataFrame
postgis_gdf = gpd.read_postgis(query, con=engine, geom_col="geometry", crs="EPSG:4326")

postgis_gdf.head()

In [ ]:
pip install duckdb -q

In [9]:
import duckdb

# Initialize a DuckDB connection
con = duckdb.connect()

In [10]:
con.query('INSTALL spatial')
con.query('LOAD spatial')

In [11]:

# Define a DuckDB SQL query
# Replace "your_table" with the table or query containing lat/lon or WKT geometry
duckdb_query = """
SELECT *, ST_AsText(ST_Point(column09, column08)) as geometry
FROM read_csv('https://s3.amazonaws.com/tripdata/202406-citibike-tripdata.zip', ignore_errors=true)
"""


In [ ]:

# Execute DuckDB query and convert to a Pandas DataFrame
duckdb_df = con.query(duckdb_query).to_df()

In [ ]:
duckdb_df.head(5)

In [13]:
duckdb_gdf = gpd.GeoDataFrame(
    duckdb_df,
    geometry=gpd.GeoSeries.from_wkt(duckdb_df["geometry"]),
    crs="EPSG:4326"
)

In [ ]:
duckdb_gdf.info()

In [ ]:
duckdb_gdf.head(100).explore()

In [17]:
nbh_query = """
SELECT *
FROM ST_Read('https://raw.githubusercontent.com/HodgesWardElliott/custom-nyc-neighborhoods/refs/heads/master/custom-pedia-cities-nyc-Mar2018.geojson')
limit 10
"""

In [ ]:
duckdb_df = con.query(nbh_query).to_df()
duckdb_df.head(5)

In [19]:
nbh_query = """
SELECT 
    count(b.column00) as count,
    n.neighborhood,
    ST_AsText(n.geom) as geom
FROM 
    ST_Read('https://raw.githubusercontent.com/HodgesWardElliott/custom-nyc-neighborhoods/refs/heads/master/custom-pedia-cities-nyc-Mar2018.geojson') n
JOIN 
    read_csv('https://s3.amazonaws.com/tripdata/202406-citibike-tripdata.zip', ignore_errors=true) b
ON 
    ST_Intersects(n.geom, ST_Point(column09, column08))
WHERE 
    CAST(column02 AS DATE) = DATE '2024-06-15'
GROUP BY 
    n.neighborhood, n.geom
"""

In [ ]:
nbh_df = con.query(nbh_query).to_df()

In [21]:
join_gdf = gpd.GeoDataFrame(
    nbh_df,
    geometry=gpd.GeoSeries.from_wkt(nbh_df["geom"]),
    crs="EPSG:4326"
)

In [ ]:
join_gdf.head()

In [ ]:
join_gdf.explore()

In [ ]:
gdf = gpd.read_parquet('data/es_cn.parquet')
gdf.info()

In [ ]:
gdf.head(100).explore()